In [1]:
import optuna
import os
from pathlib import Path
import kaggle
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, PowerTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
else:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)
    if not orig_path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

SEED = 42
SPLITS = 5
VERSION = 3
OUTPUT = f'xg_boost_clf_submisson_v{VERSION}.csv'
N_TRIALS = 30

np.random.seed(SEED)

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(path/'train.csv', index_col = 'id')
test = pd.read_csv(path/'test.csv', index_col = 'id')
orig_train = pd.read_csv(orig_path/'jm1.csv')

In [3]:
for object_features in list(orig_train.loc[:, orig_train.dtypes == 'O']):
    orig_train[object_features] = orig_train[object_features].replace({'?' : np.nan}).astype('float64')

In [4]:
X = pd.concat([train, orig_train])
y = X.pop('defects')

In [5]:
partial_pipeline = make_pipeline(SimpleImputer(), FunctionTransformer(func=np.log1p, validate=False), PowerTransformer())

In [6]:
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'n_jobs': trial.suggest_categorical('n_jobs', [-1]),
        'random_state': trial.suggest_categorical('random_state', [SEED]),
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': trial.suggest_int('n_estimators', 100, 600, 100),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    pipeline = make_pipeline(partial_pipeline, XGBClassifier(**params))
    scores = cross_val_score(pipeline, X, y, cv=SPLITS, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

[I 2023-10-16 12:54:44,050] A new study created in memory with name: no-name-b7def1c3-bc83-4cd4-8a7c-a39da47d6834
  0%|                                                                                                                                                                                                                 | 0/30 [00:00<?, ?it/s]/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be r

[I 2023-10-16 12:54:57,550] Trial 0 finished with value: 0.7850809604840705 and parameters: {'n_jobs': -1, 'random_state': 42, 'lambda': 5.438983569563615, 'alpha': 0.137440665763447, 'colsample_bytree': 0.6, 'subsample': 0.4, 'learning_rate': 0.018, 'n_estimators': 300, 'max_depth': 7, 'min_child_weight': 202}. Best is trial 0 with value: 0.7850809604840705.


/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


[I 2023-10-16 12:55:13,569] Trial 1 finished with value: 0.7845479817887575 and parameters: {'n_jobs': -1, 'random_state': 42, 'lambda': 0.01437494206988546, 'alpha': 0.13466253531825673, 'colsample_bytree': 0.4, 'subsample': 0.7, 'learning_rate': 0.014, 'n_estimators': 400, 'max_depth': 11, 'min_child_weight': 277}. Best is trial 0 with value: 0.7850809604840705.


/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


...

[I 2023-10-16 13:02:45,641] Trial 27 finished with value: 0.7863815140733784 and parameters: {'n_jobs': -1, 'random_state': 42, 'lambda': 0.04066835180694219, 'alpha': 0.04259428467448357, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.02, 'n_estimators': 400, 'max_depth': 5, 'min_child_weight': 80}. Best is trial 27 with value: 0.7863815140733784.


/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


[I 2023-10-16 13:03:06,838] Trial 28 finished with value: 0.7831916086716529 and parameters: {'n_jobs': -1, 'random_state': 42, 'lambda': 0.04397174941233527, 'alpha': 0.050020547902576595, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.02, 'n_estimators': 400, 'max_depth': 9, 'min_child_weight': 3}. Best is trial 27 with value: 0.7863815140733784.


/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


[I 2023-10-16 13:03:18,045] Trial 29 finished with value: 0.7860802348337792 and parameters: {'n_jobs': -1, 'random_state': 42, 'lambda': 0.017727432912097645, 'alpha': 0.04405732508023331, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.018, 'n_estimators': 300, 'max_depth': 7, 'min_child_weight': 75}. Best is trial 27 with value: 0.7863815140733784.


In [7]:
study.best_params, study.best_value

({'n_jobs': -1,
  'random_state': 42,
  'lambda': 0.04066835180694219,
  'alpha': 0.04259428467448357,
  'colsample_bytree': 1.0,
  'subsample': 0.6,
  'learning_rate': 0.02,
  'n_estimators': 400,
  'max_depth': 5,
  'min_child_weight': 80},
 0.7863815140733784)

In [8]:
pipeline = make_pipeline(partial_pipeline, XGBClassifier(**study.best_params))
pipeline.fit(X, y)

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('simpleimputer', SimpleImputer()),
                                 ('functiontransformer',
                                  FunctionTransformer(func=<ufunc 'log1p'>)),
                                 ('powertransformer', PowerTransformer())])),
                ('xgbclassifier',
                 XGBClassifier(alpha=0.04259428467448357, base_score=None,
                               booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=1...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None,
                               lambda=0.04066835180694219, learning_rate=0.02,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=80, missing=nan,
                               monotone_constraints=None, n_estimators=400,
                               n_jobs=-1, num_parallel_tree=None, ...))])

In [9]:
submission = test.copy()
submission['defects'] = pipeline.predict_proba(submission)[:, 1]
submission.defects.to_csv(OUTPUT)